In [9]:
import selenium, webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from datetime import datetime
import pandas as pd
import time
import re

# date: 2023/4/18

##### 爬蟲結果 #####
result = {}

## Step1. 在資料庫搜尋頁面上，爬取所有腳踏車摘要和連結

* bike_links.xlsx
    * year 年份
    * brand 腳踏車品牌
    * name 腳踏車型號
    * link 網站連結 -> step2. 爬取各型號規格時使用
    * source 資料庫來源 (mtb or rb)

In [5]:
def get_search_result(source, link):
    global driver, result
    driver.get(link)
    time.sleep(1)
    
    bike_links = driver.find_elements(By.XPATH, '//div[@class="card-footer border-0 p-2 px-2 mt-auto"]//*[@class="float-right"]//a')
    years = driver.find_elements(By.XPATH, '//div[@class="card-footer border-0 p-2 px-2 mt-auto"]//a[@class="text-dark"]')
    brands = driver.find_elements(By.XPATH, '//div[@class="card-body py-2 px-0"]//span')
    names = driver.find_elements(By.XPATH, '//div[@class="card-body py-2 px-0"]//h5')
    
    if len(bike_links) > 0:
        
        for i in range(len(bike_links)):
#             print(bike_links[i].get_attribute('href'))
            result[bike_links[i].get_attribute('href')] = {
                'year': years[i].text,
                'brand':brands[i].text,
                'name':names[i].text,
                'link':bike_links[i].get_attribute('href'),
                'source':source
            }
    
#     data_count = driver.find_element(By.XPATH, '//*[@class="ais-Stats"]').text
    if driver.find_element(By.XPATH, '//*[@class="ais-Stats"]').text != 'No Bikes':    
        btns = driver.find_elements(By.XPATH, '//*[@class="btn-group"]//a')
        if 'next'in [i.text.lower() for i in btns]:
            btn_next = btns[-2]
            get_search_result(source, btn_next.get_attribute('href'))

In [56]:
###### brands in mtb database ######
driver = webdriver.Chrome(ChromeDriverManager().install())

# 這個網站有bug，資料庫搜尋結果只會顯示最多28頁，超過則無法顯示
# 解決方法: 先以品牌做第一次篩選，但如果篩選後還是超過28頁，再用年份第二次篩選



# 搜尋結果少於28頁的品牌 -> 可以直接爬取
less_than_28 = ['Commencal', 'Cube', 'Giant', 'Scott', 'Canyon', 'Bulls', 'Cannondale', 'Merida', 'Orbea', 'Devinci', 'Kona', 'Transition', 'Yeti', 'Niner', 'YT Industries', 'Vitus', 'Fezzari', 'GT', 'Ibis', 'KHS', 'Rocky Mountain', 'Cotic', 'KTM', 'Intense', 'Radon', 'Focus', 'Ghost', 'Mondraker', 'Juliana', 'Felt', 'Fuji', 'Conway', 'Kellys', 'Propain', 'Marin', 'Nukeproof', 'NS', 'Haro', 'Orange', 'Polygon', 'BH', 'Liv', 'Diamondback', 'Lapierre', 'Evil', 'Haibike', 'BMC', 'Whyte', 'Rose', 'Bianchi', 'Kross', 'CTM', 'Bergamont', 'Jamis', 'Alchemy', 'Chromag', 'Dartmoor', 'Megamo', 'Guerrilla Gravity', 'Moustache', 'Wilier', 'Ellsworth', 'Revel', 'Salsa', 'Simplon', 'Spot', 'RSD', 'Sonder', 'Knolly', 'ROCKRIDER', 'Centurion','REEB','R RAYMON','Trailcraft','Ridley','Breezer','Silverback','Author','Corratec','Superior','Berria','MMR','Bird','Eminent','Moots','Pole','Rotwild','Framed','Mongoose','Stevens','HEAD','Stanton','On-One','Sherpa','Zerode']

# for brand in less_than_28:
#     get_search_result('mtbdatabase', f'https://mtbdatabase.com/bikes/?prod_mtbdb%5BrefinementList%5D%5Bbike_brand%5D%5B0%5D={brand}&prod_mtbdb%5Bpage%5D=1')



# 搜尋結果超過28頁的品牌 -> 需以年份第二次篩選
more_than_28 = ['Trek','Specialized','Pivot','Santa Cruz','Norco']

for brand in more_than_28:
    for year in range(2007, 2011):
        get_search_result('mtbdatabase', f'https://mtbdatabase.com/bikes/?prod_mtbdb%5BrefinementList%5D%5Bbike_brand%5D%5B0%5D={brand}&prod_mtbdb%5BrefinementList%5D%5Bbike_year%5D%5B0%5D={year}&prod_mtbdb%5Bpage%5D=1')


In [1]:
###### brands in roadbike database ######
driver = webdriver.Chrome(ChromeDriverManager().install())

roadbike_brands = ['Trek','Cannondale','Specialized','Canyon','Bianchi','Giant','Fuji','Orbea','Pegasus','Scott','Bulls','Jamis','Norco','BMC','Raleigh','Riese & Müller','Cervélo','Stevens','Origine','CUBE','Wilier','Gazelle','KHS','Devinci','Dolan','Rose','Storck 99''Diamondback','Kettler','Bergamont','GT','Schwinn','Liv','Pinarello','BH','Koga','Reid','Simplon','Lynskey','Alchemy','Litespeed','Kona','Lapierre','Kellys','Corratec','Electra','Felt','CTM','Vitus','Orro','Nordest','Van Nicholas','Cinelli','Kalkhoff','Focus','Planet X','Tern','Centurion','Moots','Author','Marin','Ridley','Genesis','Avanti','Huffy','Polygon','Rocky Mountain','Conway','Megamo','Surly','Mason','Quintana Roo','3T','Sensa','Opus','Cipollini','Kross','KTM','Kinesis','Schindelhauer','Stella','Allied','HEAD','Radon','Salsa','Whyte','R RAYMON','Aquila','Look','Obed','Bombtrack','Breezer','Cube Bikes','Silverback','Colnago','Rondo','Moustache','Raleigh Electric','Santa Cruz','Kreidler']

for brand in roadbike_brands:
    get_search_result('roadbikedatabase', f'https://roadbikedatabase.com/bikes/?prod_rbdb%5BrefinementList%5D%5Bbike_brand%5D%5B0%5D={brand}&prod_rbdb%5Bpage%5D=1')

In [7]:
result_df = pd.DataFrame.from_dict(result, orient='index').reset_index().drop('index', axis=1)
result_df

,year,brand,name,link,source
0,2023,Trek,Domane SLR Gen 3 Disc Frameset,http://127.0.0.1/bikes/2023/trek/domane/2023-t...,roadbikedatabase
1,2023,Trek,Domane SLR Gen 4 Frameset,http://127.0.0.1/bikes/2023/trek/domane/2023-t...,roadbikedatabase
2,2023,Trek,Domane SLR Frameset Gen 4,http://127.0.0.1/bikes/2023/trek/domane/2023-t...,roadbikedatabase
3,2023,Trek,Domane SLR Gen 4 Disc Frameset,http://127.0.0.1/bikes/2023/trek/domane/2023-t...,roadbikedatabase
4,2023,Trek,Domane SLR Gen 3 Frameset,http://127.0.0.1/bikes/2023/trek/domane/2023-t...,roadbikedatabase
...,...,...,...,...,...
8441,2022,Kreidler,Vitality Eco 10 Sport,http://127.0.0.1/bikes/2022/kreidler/vitality-...,roadbikedatabase
8442,2022,Kreidler,Vitality Eco 10 Cross,http://127.0.0.1/bikes/2022/kreidler/vitality-...,roadbikedatabase
8443,2022,Kreidler,Vitality Eco 10,http://127.0.0.1/bikes/2022/kreidler/vitality-...,roadbikedatabase
8444,2022,Kreidler,Raise RT4,http://127.0.0.1/bikes/2022/kreidler/raise/202...,roadbikedatabase


In [2]:
###### 各自儲存成不同檔案 ######

result_df.to_excel('bike_links_rb.xlsx', encoding='utf_8_sig', index=False)
# result_df.to_excel('bike_links_mtb.xlsx', encoding='utf_8_sig', index=False)

## Step2. 在腳踏車型號頁面上，爬取所有腳踏車資料
分成兩個part：腳踏車介紹、腳踏車規格

因為總筆數太大，先各別爬下，step3.再後續整理

* bike_overview_mtb.xlsx
    * year 年份
    * brand 腳踏車型號
    * name 腳踏車型號
    * price 價格 
    * Size 尺寸
    * Weight 重量 
    * Colour 顏色
    * specification 規格 -> step3. 依照規格分類整理
    * link 網站連結
    * source 資料庫來源

In [3]:
overview = {}

###### brands in mtb database ######
# bike_links_db = pd.read_excel('bike_links_mtb.xlsx')
# mtb_overview_db = pd.read_excel('bike_overview_mtb.xlsx')

###### brands in roadbike database ######
bike_links_db = pd.read_excel('bike_links_rb.xlsx')
rb_overview_db = pd.read_excel('bike_spec_rb.xlsx')

bike_links_db

In [4]:
# 過濾掉資料庫中，非腳踏車的資料 (可能是kit或只有frame)

for index, row in bike_links_db.iterrows():
    if str(row['name'])[-8:].lower() == 'frameset' or str(row['name'])[-10:].lower() == 'frame only' or str(row['name'])[-3:].lower() == 'kit':
        bike_links_db = bike_links_db.drop(index)
bike_links_db

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())

time.sleep(2)

for index, row in bike_links_db.iterrows():
    if row['link'] not in overview.keys() and row['link'] not in list(rb_overview_db['link']):
        driver.get(row['link'])
        time.sleep(1.25)
        
        
        #######################################################################################################
        # 從step 1.取得: year, brand, name

        overview[row['link']] = {}
        overview[row['link']]['year'] = row['year']
        overview[row['link']]['brand'] = row['brand']
        overview[row['link']]['name'] = row['name']
        
        try:
            price_element = driver.find_element(By.XPATH, '//h4[@class="price-text"]')
        except:
            pass
        else:
            overview[row['link']]['price'] = price_element.text
        
        
        #######################################################################################################
        # 爬取腳踏車介紹: Price, Size, Weight, Colour
        
        discribes = driver.find_elements(By.XPATH,'//p[@class="small font-weight-bold"]')
        for i in discribes:
            try:
                overview[row['link']][i.text.split(' / ')[0]] = i.text.split(' / ')[1]
            except:
                    print(row['link'], 'break with discribe')
        
        
        #######################################################################################################
        # 爬取腳踏車規格: 依照主分類、子分類、規格列出
        
        spec_cate = driver.find_elements(By.XPATH, '//section[@class="specifications"]//li//div[@class="col-2 px-0 px-sm-3"]')
        spec_discribe = driver.find_elements(By.XPATH, '//section[@class="specifications"]//li//div[@class="col-10 px-0 px-sm-3"]')
        
        if len(spec_cate) != len(spec_discribe):
            print(row['link'])
        else:
            for i in range(len(spec_cate)):
                overview[row['link']][spec_cate[i].text] = spec_discribe[i].text
        
        
        #######################################################################################################
        # 從step 1.取得: link, source

        overview[row['link']]['link'] = row['link']
        overview[row['link']]['source'] = row['source']

In [ ]:
# 兩個資料庫資料合併

print(len(overview.keys()))
overview_df = pd.DataFrame.from_dict(overview, orient='index').reset_index().drop('index', axis=1)
overview_df = pd.concat([overview_df, mtb_overview_db], sort=False)

In [6]:
# 有些網頁找不到頁面，可以從frame跟price是否爬取成功來判斷，失敗則刪除資料

failed = []
for index, row in overview_df.iterrows():
    if pd.isnull(row['price']) and pd.isnull(row['Frame']):
        overview_df.drop(index, inplace=True)
        print(index)

In [7]:
overview_df.to_excel('bike_overview.xlsx', encoding='utf_8_sig', index=False)

## Step3. 整理上一步驟所爬下之specification

* bike_specification.xlsx
    * ...
    * specification 規格
        * category
        * subcategory
        * specification

In [11]:
overview_df = pd.read_excel("bike_overview_rb.xlsx")
overview_df.head(1)

,year,brand,name,price,Size,Weight,Colour,specification,link,source
0,2024,Ridley,Kanzo A - Sram Apex 1x11sp HDB planbike,"$2,031.20","XXS, XS, SM, MD, LG, XL",NaN,NaN,"{'Frameset': {'Frame': 'Kanzo A Disc 7E2', 'Fo...",https://roadbikedatabase.com/bikes/2024/ridley...,roadbikedatabase


In [12]:
spec_df = pd.DataFrame()

for index, row in overview_df.iterrows():
    
    # 以dataframe型態讀取specification欄位
    double_dict = eval(row['specification'])
    
    # 把specification按照category, subcategory, spec拆分
    for category, sub_dict in double_dict.items():
        for subcategory, specification in sub_dict.items():
#             print(subcategory, specification)
#             specification_list = specification.replace(', ', ' // ').replace(' / ', ' // ').split(' // ')
#             for i in range(len(specification_list)):
            new_row = pd.DataFrame({
                'year': row['year'],
                'brand': row['brand'],
                'name': row['name'],
                'price': row['price'],
                'category': category,
                'subcategory': subcategory,
                'specification': specification,
                'source': row['source'],
                'link': row['link']
            }, index=[0])
            spec_df = pd.concat([new_row, spec_df], axis=0)
            
spec_df

,year,brand,name,price,category,subcategory,specification,source,link
0,2022,Marin,Rift Zone 29 1,"$1,899.00",Components,Headset,"FSA No 57E, Semi-Sealed Cartridge Bearings, 1 ...",mtbdatabase,http://mtbdatabase.com/bikes/2022/marin/rift-z...
0,2022,Marin,Rift Zone 29 1,"$1,899.00",Components,Grips,"Marin MTN, Closed End",mtbdatabase,http://mtbdatabase.com/bikes/2022/marin/rift-z...
0,2022,Marin,Rift Zone 29 1,"$1,899.00",Components,Stem,"Marin 3D Forged Alloy, 35mm",mtbdatabase,http://mtbdatabase.com/bikes/2022/marin/rift-z...
0,2022,Marin,Rift Zone 29 1,"$1,899.00",Components,Seatpost,"Marin Alloy, 30.9mm",mtbdatabase,http://mtbdatabase.com/bikes/2022/marin/rift-z...
0,2022,Marin,Rift Zone 29 1,"$1,899.00",Components,Saddle,Marin Speed Concept,mtbdatabase,http://mtbdatabase.com/bikes/2022/marin/rift-z...
...,...,...,...,...,...,...,...,...,...
0,2024,Ridley,Kanzo A - Sram Apex 1x11sp HDB planbike,"$2,031.20",Drivetrain,Front Derailleur,SRAM Apex 1x11sp HDB planbike,roadbikedatabase,https://roadbikedatabase.com/bikes/2024/ridley...
0,2024,Ridley,Kanzo A - Sram Apex 1x11sp HDB planbike,"$2,031.20",Drivetrain,Crank,SRAM Apex1 // 42T // 172mm,roadbikedatabase,https://roadbikedatabase.com/bikes/2024/ridley...
0,2024,Ridley,Kanzo A - Sram Apex 1x11sp HDB planbike,"$2,031.20",Wheels,Wheels,Shimano RS171 DB // Shimano 11s // Clincher,roadbikedatabase,https://roadbikedatabase.com/bikes/2024/ridley...
0,2024,Ridley,Kanzo A - Sram Apex 1x11sp HDB planbike,"$2,031.20",Frameset,Fork,4ZA Oryx Disc 12TA 7E7/MATT BLACK,roadbikedatabase,https://roadbikedatabase.com/bikes/2024/ridley...


In [2]:
spec_df.to_excel('bike_specification.xlsx', encoding='utf_8_sig', index=False)

## Step4. 合併成目標格式
* bike_spec.xlsx
    * year
    * brand
    * name
    * ... (等基本資料)
    * Headset
    * Saddle
    * Handlebar
    * ... (依照規格子分類整理)

合併完後再用excel調整子分類排列順序並標註主分類
    

In [12]:
spec_df = pd.read_excel("bike_specification.xlsx")

In [17]:
spec_df.link.isnull().sum()
spec_df.loc[spec_df.duplicated()]
spec_df = spec_df.drop_duplicates()
spec_df.head(5)

,year,brand,name,price,category,subcategory,specification,source,link
0,2000,GT,Autostream,$335.00,Components,Headset,1inch sealed mechanism,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...
1,2000,GT,Autostream,$335.00,Components,Saddle,GT Streamline,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...
2,2000,GT,Autostream,$335.00,Components,Handlebar,GT Streamline,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...
3,2000,GT,Autostream,$335.00,Drivetrain,Bottom Bracket,Sealed cartridge,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...
4,2000,GT,Autostream,$335.00,Drivetrain,Crank,"Shimano, 42 teeth",roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...


In [36]:
full_data = {}

for index, row in spec_df.iterrows():
    if row['link'] not in full_data:
        full_data[row['link']] = {
                'year': row['year'],
                'brand': row['brand'],
                'name': row['name'],
                'price': row['price'],
                'source': row['source'],
                'link': row['link']
            }

    if row['subcategory'] in full_data[row['link']].keys():
        full_data[row['link']][row['subcategory']] += row['specification']
    else:
        full_data[row['link']][row['subcategory']] = row['specification']


In [37]:
result_df = pd.DataFrame.from_dict(full_data, orient='index')
result_df.to_excel('bike_spec.xlsx', encoding='utf_8_sig', index=False)
result_df.head(10)

,year,brand,name,price,source,link,Headset,Saddle,Handlebar,Bottom Bracket,...,Brakeset,Shifters,Rear Derailleur,Front Derailleur,Tires,Spokes,Wheels,Shock,Battery,Motor
https://roadbikedatabase.com/bikes/2000/gt/autostream/2000-gt-autostream/,2000,GT,Autostream,$335.00,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...,1inch sealed mechanism,GT Streamline,GT Streamline,Sealed cartridge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/gt/autostream-ex/2000-gt-autostream-ex/,2000,GT,Autostream EX,$523.00,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/gt/aut...,1inch sealed mechanism,GT Streamline,GT Streamline,Sealed cartridge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/cannondale/c300/2000-cannondale-c300/,2000,Cannondale,C300,$576.99,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/cannon...,1 1/8inch YST,Supercush 4000,Steel hi-rise,Shimano BB-LP27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/raleigh/c400/2000-raleigh-c400/,2000,Raleigh,C400,$550.00,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/raleig...,1inch threadless Tange-Seiki RBC-2,Selle Stratos Advanced Comfort,"Aluminum, 2inch rise","Shimano cartridge, 122.5mm spindle",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/rocky-mountain/glide/2000-rocky-mountain-glide/,2000,Rocky Mountain,Glide,"$1,466.00",roadbikedatabase,https://roadbikedatabase.com/bikes/2000/rocky-...,1 1/8inch threadless Ritchey Logic,Selle Italia Onda,Ritchey Road Logic,Shimano BB-5500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/cannondale/h300/2000-cannondale-h300/,2000,Cannondale,H300,$466.99,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/cannon...,1 1/8inch threadless YST,Supercush 4000,Alico riser,Shimano cartridge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/devinci/passport/2000-devinci-passport/,2000,Devinci,Passport,$650.00,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/devinc...,1 1/8inch threadless Ritchey Scuzzy Logic,Selle Italia Avantgarde 2,Modolo aluminum,"Shimano BB-UN40, 113.5mm spindle",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/raleigh/r600/2000-raleigh-r600/,2000,Raleigh,R600,"$1,100.00",roadbikedatabase,https://roadbikedatabase.com/bikes/2000/raleig...,1inch threadless Tange Seiki DX-2,Selle Stratos Enhanced Comfort,Hsin Lung aluminum,"Shimano BB-5500, 118mm spindle",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/raleigh/r800/2000-raleigh-r800/,2000,Raleigh,R800,"$1,960.00",roadbikedatabase,https://roadbikedatabase.com/bikes/2000/raleig...,1inch threadless Tange Seiki DX-2,Selle Italia Trimatica,Cinelli Touch-E,"Shimano BB-5500, 110mm spindle",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://roadbikedatabase.com/bikes/2000/raleigh/sc-auto/2000-raleigh-sc-auto/,2000,Raleigh,SC-Auto,$440.00,roadbikedatabase,https://roadbikedatabase.com/bikes/2000/raleig...,1inch Victor VP-H67,Selle Stratos Enhanced Comfort,"Steel, Rise","Victor VPB-33, 120mm spindle",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
